In [ ]:
import psutil
import pandas
import re
import requests
import json
import os
import psutil
import gmaps
import gmaps.datasets
import math

In [ ]:
privIP = re.compile("^(?:10|127|172\.(?:1[6-9]|2[0-9]|3[01])|192\.168)\..*")
priv6 = re.compile("::1")
url_str = 'https://ipinfo.io/'
token_str = '?token=xxxxxxxx' #replace xxxxx with ipInfo api token

In [ ]:
netstat = pandas.DataFrame(columns=['status', 'pid', 'local_ip', 'local_port', 'remote_ip', 'remote_port'])
con = psutil.net_connections()
for item in con:
    try:
        if not re.search(privIP, item.raddr.ip) and not re.search(priv6, item.raddr.ip):
            #print(item.status, item.pid, item.laddr.ip, item.laddr.port, item.raddr.ip, item.raddr.port)
            data = {'status':item.status, 'pid':item.pid, 'local_ip':item.laddr.ip, 'local_port': item.laddr.port, 'remote_ip':item.raddr.ip, 'remote_port':item.raddr.port}
            s = pandas.Series(data)
            netstat = netstat.append(s, ignore_index=True)
    except:
        pass

In [ ]:
ipInfo = pandas.DataFrame(columns=['ip','hostname','city','region','country','loc','org','postal','timezone'])
for ip in netstat.remote_ip:
    #print(requests.get(url_str + ip + token_str).text)
    data = (json.loads(requests.get(url_str + ip + token_str).text))
    s = pandas.Series(data)
    ipInfo = ipInfo.append(s, ignore_index = True)

In [ ]:
# Iterate over all running process
processes = pandas.DataFrame(columns=['process_name', 'pid'])
for proc in psutil.process_iter():
    try:
        #print(proc.name(), proc.pid)
        data = {'process_name':proc.name(), 'pid':proc.pid}
        s = pandas.Series(data)
        processes = processes.append(s, ignore_index = True)
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass

In [ ]:
mergedf = pandas.merge(netstat, ipInfo.rename(columns={'ip':'remote_ip'}), how='outer')
mergedf = pandas.merge(mergedf, processes, how='outer')
processConnections = mergedf[mergedf.remote_ip.notnull()]

In [ ]:
googleAPIKey = '' #add google api key here
gmaps.configure(api_key=googleAPIKey)

In [ ]:
locList = list(mergedf['loc'].dropna())

locations = []
for loc in locList:
    locations.append(eval(loc))
for loc in locations:
    for s in loc:
        s = float(s)


In [ ]:
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations))
fig

In [ ]:
pandas.set_option('display.max_rows', None)
processConnections